# Set up

In [1]:
project_folder = "/content/drive/MyDrive/water_quality_predictor"

from google.colab import drive
drive.mount('/content/drive')

%cd "{project_folder}"
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/water_quality_predictor
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 301 bytes | 1024 bytes/s, done.
From https://github.com/zalexdevon/water_quality_predictor
   8f4bf2f..b262210  main       -> origin/main
Updating 8f4bf2f..b262210
Fast-forward
 run.py | 3 ++-
 1 file changed, 2 insertions(+), 1 deletion(-)


# Cài đặt môi trường

In [2]:
!git pull
!pip install -r requirements.txt

Updating 6ee5793..8f4bf2f
error: Your local changes to the following files would be overwritten by merge:
	src/classifier/components/data_correction.py
	src/classifier/components/data_transformation.py
	src/classifier/components/data_transformation_batch.py
	src/classifier/components/data_transformation_batch_weighted.py
	src/classifier/components/data_transformation_weighted.py
	src/classifier/config/configuration.py
	src/classifier/data_correction_code/dc3_new2.py
	src/classifier/entity/config_entity.py
Please commit your changes or stash them before you merge.
Aborting
Obtaining file:///content/drive/MyDrive/water_quality_predictor (from -r requirements.txt (line 17))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of imbalanced-learn to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

# Thư viện

In [ ]:
import pandas as pd
import numpy as np
from Mylib import myfuncs
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import plotly.express as px
import os
from src.data_correction import dc1, dc

np.set_printoptions(suppress=True)

# Đọc dữ liệu


In [ ]:
df = myfuncs.load_python_object("artifacts/data_ingestion/train_data.pkl")

df.head()


,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,Copper,...,Sulfate,Conductivity,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Target,taken_time
116899,7.034305,3.510000e-14,2.859233,132.125438,6.540000e-235,2.152380,Light Yellow,1.377299,1.551360,0.095312,...,335.357965,344.015095,3.756497,0.000332,504.928844,Reservoir,7.116495,72.255851,1,1900-01-01
155434,6.599581,3.990000e-07,5.371393,199.138779,1.330000e-58,1.552503,Near Colorless,0.071201,1.776722,0.420041,...,199.562148,421.907595,3.922976,0.021912,403.644429,Spring,23.035464,26.433255,0,1900-01-01
165968,7.670866,9.310000e-06,1.156363,245.548989,7.130000e-66,0.608690,Colorless,0.428635,0.506733,0.234863,...,79.666497,499.280918,3.755862,0.002062,391.642190,Spring,5.728718,54.360327,0,1900-01-01
150476,8.024941,8.664510e-02,3.146721,174.001989,2.460000e-113,1.151119,Colorless,0.583815,0.543565,0.169842,...,112.425132,304.724172,3.713097,0.002520,53.432049,River,26.853255,71.579596,0,1900-01-01
255559,6.738499,3.550000e-11,3.481104,225.121948,5.470000e-05,1.494975,Faint Yellow,0.863283,1.961606,NaN,...,103.405765,377.043871,1.744797,0.000029,157.238713,Well,9.778199,66.545395,0,1900-01-01


In [ ]:
df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Target', 'taken_time'],
      dtype='object')

In [ ]:
df['Source'].unique()

array(['Reservoir', 'Spring', 'River', 'Well', 'Lake', 'Ground', nan,
       'Stream', 'Aquifer'], dtype=object)

# Ý nghĩa các cột

| Cot                       | Y nghia                                                               |  Phan loai |
| ------------------------- | --------------------------------------------------------------------- | --------- |
| pH                |   độ PH                              |  num   |
| Iron                |  nồng độ Sắt                             |  num   |
| Nitrate                |  nồng độ Nitrat                           |  num   |
| Chloride                |  nồng độ ion âm Clo                        |  num   |
| Lead                |  nồng độ chì                        |  num   |
| Zinc                |  nồng độ kẽm                     |  num   |
| Color                |  màu sắc của nước                    |  ord   |
| Turbidity                |  độ đục của nước                    |  num   |
| Fluoride                |  nồng độ của Flo                |  num   |
| Copper                |  nồng độ của đồng             |  num   |
| Odor                |  cảm giác ngửi của mũi về nước           |  num   |
| Sulfate                |  nồng độ sunfat          |  num   |
| Conductivity                |  độ dẫn điện         |  num   |
| Chlorine                |  nồng độ của khí Clo        |  num   |
| Manganese                |  nồng độ của mangan        |  num   |
| Total Dissolved Solids                |  tổng chất rắn hòa tan        |  num   |
| Source           |  nguồn nước   |  nom   |
| Water Temperature           | nhiệt độ nước   |  num   |
| Air Temperature       | nhiệt độ không khí  |  num   |
| Target       | có dơ hay không  |  target   |
| **taken_time**       | thời gian lấy mẫu  |  date_time   |

# Xóa các cột không liên quan

In [ ]:
df = df.drop(
    columns=[
        "taken_time",
    ]
)

df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Target'],
      dtype='object')

# Xóa các cột có tỉ lệ missing lớn

In [ ]:
null_percent = df.isnull().mean() * 100
null_percent = null_percent.sort_values(ascending=False)
null_percent


,0
Copper,3.310748
Sulfate,3.265541
Fluoride,3.141639
Odor,3.027225
Chloride,2.928996
Water Temperature,2.830768
Conductivity,2.720819
Zinc,2.538315
pH,1.918805
Manganese,1.877505


Tỉ lệ missing của các cột đều khá nhỏ -> Không xóa cột nào hết


In [ ]:
df.shape


(179174, 20)

# Đổi tên cột

In [ ]:
df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Target'],
      dtype='object')

In [ ]:
rename_dict = {
    "pH": "pH_num",
    "Iron": "Iron_num",
    "Nitrate": "Nitrate_num",
    "Chloride": "Chloride_num",
    "Lead": "Lead_num",
    "Zinc": "Zinc_num",
    "Color": "Color_ord",
    "Turbidity": "Turbidity_num",
    "Fluoride": "Fluoride_num",
    "Copper": "Copper_num",
    "Odor": "Odor_num",
    "Sulfate": "Sulfate_num",
    "Conductivity": "Conductivity_num",
    "Chlorine": "Chlorine_num",
    "Manganese": "Manganese_num",
    "Total Dissolved Solids": "Total_Dissolved_Solids_num",
    "Source": "Source_nom",
    "Water Temperature": "Water_Temperature_num",
    "Air Temperature": "Air_Temperature_num",
    "Target": "Target_target",

}


df = df.rename(columns=rename_dict)

df.columns


Index(['pH_num', 'Iron_num', 'Nitrate_num', 'Chloride_num', 'Lead_num',
       'Zinc_num', 'Color_ord', 'Turbidity_num', 'Fluoride_num', 'Copper_num',
       'Odor_num', 'Sulfate_num', 'Conductivity_num', 'Chlorine_num',
       'Manganese_num', 'Total_Dissolved_Solids_num', 'Source_nom',
       'Water_Temperature_num', 'Air_Temperature_num', 'Target_target'],
      dtype='object')

# Sắp xếp các cột theo đúng thứ tự

In [ ]:
numeric_cols, numericCat_cols, cat_cols, binary_cols, nominal_cols, ordinal_cols, target_col = myfuncs.get_different_types_cols_from_df_4(df)


df = df[
    numeric_cols
    + numericCat_cols
    + binary_cols
    + nominal_cols
    + ordinal_cols
    + [target_col]
]


df.head()


,pH_num,Iron_num,Nitrate_num,Chloride_num,Lead_num,Zinc_num,Turbidity_num,Fluoride_num,Copper_num,Odor_num,Sulfate_num,Conductivity_num,Chlorine_num,Manganese_num,Total_Dissolved_Solids_num,Water_Temperature_num,Air_Temperature_num,Source_nom,Color_ord,Target_target
116899,7.034305,3.510000e-14,2.859233,132.125438,6.540000e-235,2.152380,1.377299,1.551360,0.095312,2.927038,335.357965,344.015095,3.756497,0.000332,504.928844,7.116495,72.255851,Reservoir,Light Yellow,1
155434,6.599581,3.990000e-07,5.371393,199.138779,1.330000e-58,1.552503,0.071201,1.776722,0.420041,1.569571,199.562148,421.907595,3.922976,0.021912,403.644429,23.035464,26.433255,Spring,Near Colorless,0
165968,7.670866,9.310000e-06,1.156363,245.548989,7.130000e-66,0.608690,0.428635,0.506733,0.234863,2.942056,79.666497,499.280918,3.755862,0.002062,391.642190,5.728718,54.360327,Spring,Colorless,0
150476,8.024941,8.664510e-02,3.146721,174.001989,2.460000e-113,1.151119,0.583815,0.543565,0.169842,0.581440,112.425132,304.724172,3.713097,0.002520,53.432049,26.853255,71.579596,River,Colorless,0
255559,6.738499,3.550000e-11,3.481104,225.121948,5.470000e-05,1.494975,0.863283,1.961606,NaN,2.915724,103.405765,377.043871,1.744797,0.000029,157.238713,9.778199,66.545395,Well,Faint Yellow,0


# Kiểm tra kiểu dữ liệu các cột

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 179174 entries, 116899 to 167276
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   pH_num                      175736 non-null  float64
 1   Iron_num                    177949 non-null  float64
 2   Nitrate_num                 175925 non-null  float64
 3   Chloride_num                173926 non-null  float64
 4   Lead_num                    178372 non-null  float64
 5   Zinc_num                    174626 non-null  float64
 6   Turbidity_num               177760 non-null  float64
 7   Fluoride_num                173545 non-null  float64
 8   Copper_num                  173242 non-null  float64
 9   Odor_num                    173750 non-null  float64
 10  Sulfate_num                 173323 non-null  float64
 11  Conductivity_num            174299 non-null  float64
 12  Chlorine_num                177402 non-null  float64
 13  Manganese_num 

Các cột từ 17 trở đi bị sai dữ liệu


## Chuyển kdl = kdl mong muốn + NAN


In [ ]:
for col in df.columns.tolist()[17:]:
  print(f"{col} -> {set(map(type, df[col]))}")


Source_nom -> {<class 'float'>, <class 'str'>}
Color_ord -> {<class 'float'>, <class 'str'>}
Target_target -> {<class 'int'>}


In [ ]:
for col_name in ['Source_nom', 'Color_ord']:
  float_values = df[col_name][df[col_name].apply(lambda item: isinstance(item, float))].unique().tolist()
  print(f"{col_name} -> {float_values}")


Source_nom -> [nan]
Color_ord -> [nan]


In [ ]:
df['Target_target'].unique()

array([1, 0])

Tất cả các cột đều thỏa

# Kiểm tra nội dung các cột `binary`


In [ ]:
for col in binary_cols:
  print(f"{col} -> {df[col].unique().tolist()}")

Không có cột nào hết


# Kiểm tra nội dung các cột `nominal`


In [ ]:
for col in nominal_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Source_nom -> ['Reservoir', 'Spring', 'River', 'Well', 'Lake', 'Ground', nan, 'Stream', 'Aquifer']


# Kiểm tra nội dung các cột `ordinal`


In [ ]:
for col in ordinal_cols:
    print(f"{col} -> {df[col].unique().tolist()}")


Color_ord -> ['Light Yellow', 'Near Colorless', 'Colorless', 'Faint Yellow', 'Yellow', nan]


# Kiểm tra nội dung các cột `target`


In [ ]:
print(f"{target_col} -> {df[target_col].unique().tolist()}")


Target_target -> [1, 0]


# Fill missing value


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="mean"), numeric_cols),
        ("numCat", SimpleImputer(strategy="most_frequent"), numericCat_cols),
        ("cat", SimpleImputer(strategy="most_frequent"), cat_cols),
        ("target", SimpleImputer(strategy="most_frequent"), [target_col]),
    ]
)

df = preprocessor.fit_transform(df)

df = pd.DataFrame(
    df, columns=numeric_cols + numericCat_cols + cat_cols + [target_col]
)

df.head()


,pH_num,Iron_num,Nitrate_num,Chloride_num,Lead_num,Zinc_num,Turbidity_num,Fluoride_num,Copper_num,Odor_num,Sulfate_num,Conductivity_num,Chlorine_num,Manganese_num,Total_Dissolved_Solids_num,Water_Temperature_num,Air_Temperature_num,Source_nom,Color_ord,Target_target
0,7.034305,0.0,2.859233,132.125438,0.0,2.15238,1.377299,1.55136,0.095312,2.927038,335.357965,344.015095,3.756497,0.000332,504.928844,7.116495,72.255851,Reservoir,Light Yellow,1
1,6.599581,0.0,5.371393,199.138779,0.0,1.552503,0.071201,1.776722,0.420041,1.569571,199.562148,421.907595,3.922976,0.021912,403.644429,23.035464,26.433255,Spring,Near Colorless,0
2,7.670866,0.000009,1.156363,245.548989,0.0,0.60869,0.428635,0.506733,0.234863,2.942056,79.666497,499.280918,3.755862,0.002062,391.64219,5.728718,54.360327,Spring,Colorless,0
3,8.024941,0.086645,3.146721,174.001989,0.0,1.151119,0.583815,0.543565,0.169842,0.58144,112.425132,304.724172,3.713097,0.00252,53.432049,26.853255,71.579596,River,Colorless,0
4,6.738499,0.0,3.481104,225.121948,0.000055,1.494975,0.863283,1.961606,0.492943,2.915724,103.405765,377.043871,1.744797,0.000029,157.238713,9.778199,66.545395,Well,Faint Yellow,0


# Chuyển đổi các cột về đúng kiểu dữ liệu


In [ ]:
df[numeric_cols] = df[numeric_cols].astype("float32")
df[numericCat_cols] = df[numericCat_cols].astype("float32")
df[cat_cols] = df[cat_cols].astype("category")
df[target_col] = df[target_col].astype("category")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179174 entries, 0 to 179173
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   pH_num                      179174 non-null  float32 
 1   Iron_num                    179174 non-null  float32 
 2   Nitrate_num                 179174 non-null  float32 
 3   Chloride_num                179174 non-null  float32 
 4   Lead_num                    179174 non-null  float32 
 5   Zinc_num                    179174 non-null  float32 
 6   Turbidity_num               179174 non-null  float32 
 7   Fluoride_num                179174 non-null  float32 
 8   Copper_num                  179174 non-null  float32 
 9   Odor_num                    179174 non-null  float32 
 10  Sulfate_num                 179174 non-null  float32 
 11  Conductivity_num            179174 non-null  float32 
 12  Chlorine_num                179174 non-null  float32 
 13 

In [ ]:
df['Target_target'].cat.categories

Index([0, 1], dtype='int64')

# Loại bỏ duplicates


In [ ]:
print(f"Tỉ lệ duplicates: {len(df.index[df.duplicated()]) / len(df.index) * 100}")
df = df.drop_duplicates().reset_index(drop=True)

Tỉ lệ duplicates: 0.45877192003304046


In [ ]:
df.shape

(178352, 20)

# Chuẩn bị thứ tự cho các cột `ordinal`, `binary`


In [ ]:
binary_cols + ordinal_cols

['Color_ord']

In [ ]:
a = df['Color_ord'].unique().tolist()
a

['Light Yellow', 'Near Colorless', 'Colorless', 'Faint Yellow', 'Yellow']

In [ ]:
feature_ordinal_dict = {
    "Color_ord": ['Colorless','Near Colorless', 'Faint Yellow',  'Light Yellow','Yellow'],
}

# Các tham số

In [ ]:
data_correction_name = "dc1"

# Run

In [ ]:
# Các tham số 
train_data_path = "artifacts/data_ingestion/train_data.pkl"
data_correction_path = f"artifacts/data_correction/{data_correction_name}"

# Tạo thư mục
myfuncs.create_directories([data_correction_path])

# Load data 
df_train = myfuncs.load_python_object(train_data_path)    

# Tạo transformer
transformer = Pipeline(
    steps=[
        ("1", dc1.BeforeTransformer()),
        ("2", dc1.MissingValueHandler()),
        ("3", dc1.AfterTransformer()),
    ]
)

# Correct data 
df_train_transformed = transformer.fit_transform(df_train)

# Lưu data
dc.save_data_for_data_correction(data_correction_path, transformer, df_train_transformed, feature_ordinal_dict)
